## Librerias

In [1]:
from werkzeug.wrappers import Request, Response

import collections
from amigocloud import AmigoCloud

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


## Conectar a AmigoCloud

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [4]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Buscar registros de soca

In [5]:
semana = 14

In [6]:
query = {'query': 'select insp.canhero, replace(insp.propiedad, \'_\', \' \') propiedad, insp.user, soca.observacion, gal.s3_filename from dataset_302136 soca\
         inner join dataset_302115 insp on insp.amigo_id=soca.insp_ref_soca\
         inner join gallery_45144 gal on soca.amigo_id=gal.source_amigo_id\
         where gal.s3_filename is not null and date_part(\'week\', soca.fecha_de_registro)={sem}'.format(sem=semana)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32424/sql', query)
data = select['data']
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [7]:
data

[{'canhero': '2004 / CLAROS TORRICO CASIANO',
  'propiedad': '645 / P-044 COM.CAMP.FAJA 27 DE MAYO',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Buen control de maleza',
  's3_filename': '45144_20230405155117207.jpg'},
 {'canhero': '2004 / CLAROS TORRICO CASIANO',
  'propiedad': '640 / COM.CAMP.SAN JUAN DE LOS AMARILLOS (PARCELA 013)--CLAROS',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Bremura en la orilla, buen control de maleza ',
  's3_filename': '45144_20230405161726822.jpg'}]

In [9]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate(ruta + "/templates/tpl_soca_fotos.docx")

lista_fotos_inline = []
for foto in data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({
        'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['s3_filename'], width=Mm(70)), 
        'canhero': foto['canhero'],
        'propiedad': foto['propiedad'],
        'obs' : foto['observacion']
    })
    print(foto['s3_filename'])

context = {'fotos':lista_fotos_inline}

doc.render(context)
file_name = 'SOCA SEMANA - ' + str(semana)
doc.save(ruta + '/_' + file_name + '.docx')

45144_20230405155117207.jpg
45144_20230405161726822.jpg
